In [1]:
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

    Updating registry at `C:\Users\44780\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [7]:
### 3. Create solver function (dcopf)

In [8]:
gens

LoadError: UndefVarError: `gens` not defined

In [30]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode gives 1...150
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))  #gives 36 nodes 1...36
    
    #set of lines
    L = lines.id #each line has been assigned an id from 1...66 node is
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100000000 # base MVA is 100 MVA for this system
    
    # Decision variables   #ammended to include Ug
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[L]          # flows along each line updated
        #U[G]              #Variable Ug    
    end)
    
    #set_binary(U[G])
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # old Supply demand balances
    #@constraint(DCOPF, cBalance[i in N], 
     #   sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
      #      + sum(load for load in loads[loads.connnode .== i,:demand]) 
       # == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    #)

    # demand balance
    @constraint(DCOPF, cSupBalance[i in N],
        sum((GEN[g]) for g in gens[gens.connode .== i, :id]) 
        - sum((FLOW[l]) for l in lines[lines.fromnode .== i, :fromnode])
        == loads[loads.connnode .== i,:demand]
    )
            
    #demand balance reattempt? # need to sum over set G_i, sum over  set L_i
    #@constraint(DCOPF, cSBal,
     #       sum((I[g,N])*GEN[g]) for g in gens[])
      #      -sum((A[l,N])*FLOW[l] for l in lines[lines.id .== l])

    # Max generation constraint # AMMENDED 
    #@constraint(DCOPF, cMaxGen[g in G],
     #         gens[g,:pgmin]*U[G] <= GEN[g] <= gens[g,:pgmax]*U[G])
            
     # Max generation constraint # AMMENDED 
    @constraint(DCOPF, cMaxGen[g in G],
              GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch #AMMENDED
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode]] == 
            lines[l,:b] * sum((THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])) 
            
    )
    
    # line flow constraints AMMENDED
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            -lines[l,:capacity] <= FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    #Ammmended to show flows on each line 
    flows = DataFrame(
        flow = lines.b .* sum((angles[lines.fromnode] .- 
                        angles[lines.tonode]) * A[L,N] ))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
   
    # Return load payment
    #loadpay = sum((dual.(cSupBalance)) * loads[loads.connnode .== i,:demand])
    
    #congestion cost
    #congestc = -sum(FLOW[l] for l in load )

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [2]:
datadir = joinpath("ReducedNetwork") 
gens = CSV.read(joinpath(datadir,"GenRedNet2.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"TransRedNet.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"demandRedNet.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines.fromnode
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

lines.capacity = 0.4*lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
#loads = loads[:,["connnode","interval-1_load"]]
#rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Float64,Int64,Float64
1,1,2,0.0122,0.02,1,4000.0,1,50.0
2,1,3,0.007,0.15,1,4000.0,2,6.66667
3,1,2,0.0122,0.02,1,4000.0,3,50.0
4,1,3,0.007,0.15,1,4000.0,4,6.66667
5,2,4,0.0004,0.065,1,4000.0,5,15.3846
6,2,4,0.0004,0.065,1,4000.0,6,15.3846
7,4,7,0.00211,0.0135,1,4000.0,7,74.0741
8,4,6,0.0013,0.023,1,4000.0,8,43.4783
9,4,6,0.0013,0.023,1,4000.0,9,43.4783


In [3]:
gens

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64
1,1,0,0,12.0,668.0,0,100,-100,0,1
2,1,0,0,395.37,554.28,0,100,-100,0,2
3,1,0,0,86.38,300.0,0,100,-100,0,3
4,1,0,0,32.0,18.72,0,100,-100,0,4
5,2,0,0,37.0,50.6,0,100,-100,0,5
6,2,0,0,400.68,1524.0,0,100,-100,0,6
7,2,0,0,400.68,12.0,0,100,-100,0,7
8,3,0,0,32.0,550.99,0,100,-100,0,8
9,3,0,0,27.0,19.0,0,100,-100,0,9


In [4]:
loads

Row,connnode,demand
,Int64,Float64
1,1,-468.0
2,2,-513.0
3,3,-555.0
4,4,-1308.0
5,5,-502.0
6,6,-1176.0
7,7,-745.0
8,8,-117.5
9,9,-130.0


In [6]:
solution = dcopf_ieee(gens, lines, loads);

LoadError: KeyError: key 30 not found

In [37]:
solution.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,2,1138.23
6,2,1138.23
7,2,1138.23
8,3,0.0
9,3,0.0


In [13]:
df4 = solution.prices

LoadError: UndefVarError: `solution` not defined

In [14]:
solution.angles

LoadError: UndefVarError: `solution` not defined

In [35]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
    
      # Set of all nodes
    N = sort(union((lines.fromnode), 
            (lines.tonode))) #took out unique
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100000000 # base MVA is 100 MVA for this system *LOOK INTO THIS FURTHER*
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows along each line
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))

    #load payment
    #loadpay = sum(dual.(cBalance)*(loads for loads in loads[loads.connnode, ;demand]))
    

    #congestion cost

    #Total Gen prof
    #genprof = sum(GEN[g] for g in gens[gens.connnode .== i,:id]) * sum()
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        #loadpay,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [21]:
sum(loads for loads in loads[loads.connnode, :demand]) #VERIFY SUM of loads for load payment

-56325.86

In [7]:
#NationalPricing attempt
#Function to solve DC F problem using IEEE test cases
#Inputs:
    #gen_info -- dataframe with generator info
    #line_info -- dataframe with transmission lines info
    #loads  -- dataframe with load info
#
function np_attempt(gens, lines, loads)
    NP = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
    
      # Set of all nodes
    N = sort(union((lines.fromnode), 
            (lines.tonode))) #took out unique
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system *LOOK INTO THIS FURTHER*
    
    # Decision variables   
    @variables(NP, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows along each line
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    #fix(THETA[1],0)
                
    # Objective function
    @objective(NP, Min, 
        sum(gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(NP, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(NP, cMaxGen[g in G],
                   GEN[g] <= gens[g,:pgmax])
    
 
    


    # Solve statement (! indicates runs in place)
    optimize!(NP)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))

    #load payment
    #loadpay = sum(dual.(cBalance)*(loads for loads in loads[loads.connnode, :demand]))
    

    #congestion cost
    
    
    #Total Gen prof
    #genprof = sum(GEN[g] for g in gens[gens.connnode .== i,:id]) * sum()
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        #loadpay,
        cost = objective_value(NP),
        status = termination_status(NP)
    )
end

np_attempt (generic function with 1 method)

In [8]:
sol = np_attempt(gens, lines, loads);


Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 66 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-95); columns 0(-936); elements 0(-232) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.01


In [9]:
sol.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,2,0.0
6,2,0.0
7,2,0.0
8,3,0.0
9,3,0.0


In [11]:
gendf = sol.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,2,0.0
6,2,0.0
7,2,0.0
8,3,0.0
9,3,0.0


In [24]:
sum(gendf[:,2])

0.0

In [23]:
sum(sol.prices[:,2])

0.0

In [17]:
sol.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,0.0
2,1,3,0.0
3,1,2,0.0
4,1,3,0.0
5,2,4,0.0
6,2,4,0.0
7,4,7,0.0
8,4,6,0.0
9,4,6,0.0


In [22]:
sum(flowsdf[:,3])

0.0

In [5]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
       
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )

    #supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
# We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [20]:
df2 = solution.flows

LoadError: UndefVarError: `solution` not defined

CSV.write("C:\\Users\\44780\\OneDrive\\Documents\\FYP-Modelling\\testcase\\Flows.csv", )

In [21]:
CSV.write("C:\\Users\\44780\\OneDrive\\Documents\\FYP-Modelling\\testcase\\Flows.csv", df2 )

"C:\\Users\\44780\\OneDrive\\Documents\\FYP-Modelling\\testcase\\Flows.csv"

In [39]:
CSV.write("C:\\Users\\44780\\OneDrive\\Documents\\FYP-Modelling\\testcase\\Prices.csv", df4 )

"C:\\Users\\44780\\OneDrive\\Documents\\FYP-Modelling\\testcase\\Prices.csv"

In [21]:
loadsts = CSV.read(joinpath(datadir,"ts_load.csv"), DataFrame);

In [22]:
loadsts

Row,datetime,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2015-01-01 00:00:00,1191.27,1014.67,513.505,780.383,995.278,728.096,784.74,2146.38,1146.46,287.011,277.552,888.05,1420.79,944.495,1511.58,107.854,94.6525,636.591,378.019,1216.33,809.892,394.444,1154.56,334.507,419.964,99.7062,982.691,329.622,220.23,34.7211,219.528,238.912,266.55,578.812,350.294,483.744
2,2015-01-01 01:00:00,1170.09,996.626,504.371,766.502,977.575,715.146,770.782,2108.2,1126.07,281.906,272.616,872.254,1395.52,927.695,1484.69,105.936,92.9689,625.268,371.295,1194.7,795.487,387.428,1134.03,328.557,412.494,97.9327,965.211,323.759,216.313,34.1035,215.623,234.663,261.809,568.517,344.063,475.14
3,2015-01-01 02:00:00,1093.24,931.172,471.246,716.161,913.372,668.178,720.16,1969.74,1052.12,263.391,254.711,814.968,1303.87,866.768,1387.18,98.9783,86.863,584.203,346.91,1116.23,743.242,361.983,1059.55,306.978,385.403,91.5009,901.82,302.496,202.106,31.8637,201.462,219.251,244.614,531.179,321.466,443.934
4,2015-01-01 03:00:00,1025.55,873.514,442.067,671.817,856.816,626.804,675.568,1847.78,986.969,247.082,238.94,764.505,1223.13,813.098,1301.29,92.8497,81.4845,548.029,325.43,1047.12,697.221,339.569,993.943,287.97,361.539,85.8352,845.98,283.766,189.592,29.8908,188.987,205.675,229.468,498.288,301.561,416.446
5,2015-01-01 04:00:00,965.161,822.081,416.038,632.26,806.366,589.898,635.79,1738.98,928.855,232.534,224.871,719.491,1151.11,765.222,1224.67,87.3826,76.6866,515.761,306.268,985.463,656.168,319.575,935.419,271.014,340.251,80.7811,796.168,267.057,178.428,28.1308,177.86,193.565,215.957,468.949,283.805,391.926
6,2015-01-01 05:00:00,942.238,802.556,406.156,617.243,787.214,575.887,620.69,1697.68,906.794,227.011,219.53,702.402,1123.77,747.047,1195.58,85.3072,74.8653,503.511,298.994,962.057,640.583,311.985,913.202,264.578,332.17,78.8625,777.258,260.715,174.191,27.4626,173.635,188.968,210.828,457.811,277.064,382.617
7,2015-01-01 06:00:00,965.622,822.474,416.237,632.562,806.752,590.18,636.094,1739.81,929.299,232.645,224.978,719.835,1151.67,765.588,1225.25,87.4244,76.7233,516.007,306.414,985.934,656.482,319.728,935.866,271.144,340.414,80.8198,796.548,267.185,178.514,28.1442,177.945,193.657,216.06,469.173,283.941,392.113
8,2015-01-01 07:00:00,982.43,836.79,423.482,643.572,820.794,600.452,647.166,1770.09,945.475,236.694,228.894,732.364,1171.71,778.914,1246.58,88.9461,78.0588,524.989,311.748,1003.1,667.909,325.293,952.156,275.864,346.339,82.2265,810.413,271.836,181.621,28.6341,181.042,197.028,219.821,477.339,288.883,398.938
9,2015-01-01 08:00:00,1016.24,865.586,438.055,665.719,849.04,621.116,669.437,1831.01,978.011,244.84,236.771,757.567,1212.03,805.718,1289.48,92.007,80.745,543.055,322.476,1037.61,690.893,336.487,984.922,285.357,358.258,85.0562,838.302,281.19,187.871,29.6195,187.272,203.808,227.385,493.766,298.824,412.667


In [23]:
loadsts.id = 1:nrow(loadsts);

In [24]:
#loads ids for defiining set T
loadsts.id

8760-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 8749
 8750
 8751
 8752
 8753
 8754
 8755
 8756
 8757
 8758
 8759
 8760

In [20]:
linestest = CSV.read(joinpath(datadir,"linetest4.csv"), DataFrame);

In [21]:
linestest

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.00266925,0.0323779,1,10000,1,0.0196113
2,1,4,0.00287887,0.0286089,1,10000,2,0.0103084
3,2,3,0.00131306,0.0190229,1,10000,3,0.0102966
4,2,4,0.000803188,0.0149712,1,10000,4,0.00476255
5,3,1,0.000399063,0.00548875,1,10000,5,0.00238071
6,4,3,0.00114981,0.0183772,1,10000,6,0.0126698


In [7]:
genstest = CSV.read(joinpath(datadir,"GentsTest.csv"), DataFrame);

In [8]:
genstest

Row,1,2,3,4
,Float64,Float64,Float64,Float64
1,46.3057,65.8899,85.4789,39.3055
2,46.3057,65.8899,85.4789,39.3055
3,46.3057,65.8899,85.4789,39.3055
4,46.3057,65.8899,85.4789,39.3055
5,46.3057,65.8899,85.4789,39.3055
6,46.3057,65.8899,85.4789,39.3055
7,46.3057,65.8899,85.4789,39.3055
8,46.3057,65.8899,85.4789,39.3055
9,46.3057,65.8899,85.4789,39.3055


In [10]:
loadtest = CSV.read(joinpath(datadir,"loadtstest.csv"), DataFrame);

In [11]:
loadtest

Row,1,2,3,4
,Float64?,Float64?,Float64?,Float64?
1,1191.27,1014.67,513.505,780.383
2,1170.09,996.626,504.371,766.502
3,1093.24,931.172,471.246,716.161
4,1025.55,873.514,442.067,671.817
5,965.161,822.081,416.038,632.26
6,942.238,802.556,406.156,617.243
7,965.622,822.474,416.237,632.562
8,982.43,836.79,423.482,643.572
9,1016.24,865.586,438.055,665.719


In [22]:
sort(union(unique(linestest.fromnode), 
            unique(linestest.tonode)))

4-element Vector{Int64}:
 1
 2
 3
 4

In [27]:
sum(genstest)

LoadError: AbstractDataFrame is not iterable. Use eachrow(df) to get a row iterator or eachcol(df) to get a column iterator

In [ ]:
gens = CSV.read(joinpath(datadir,"Gen36.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran36_b_csv.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load36_csv.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
genstest.id = 1:ncol(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines.fromnode
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

lines.capacity = 0.4*lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

In [ ]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = ncol(genstest) #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode gives 1...150
    
      # Set of all nodes
    N = sort(union(unique(linestest.fromnode), 
            unique(linestest.tonode)))  #gives 36 nodes 1...36
    
    #set of lines
    L = linestest.id #each line has been assigned an id from 1...66 node is
    
    #definition of timeset T
    T = loadsts.id
    
    
    
    # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100000000 # base MVA is 100 MVA for this system
    
    # Decision variables   #ammended to include Ug
    @variables(DCOPF, begin
        GEN[G, T]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[L, T]          # flows along each line updated
        #U[G]              #Variable Ug    
    end)
    
    #set_binary(U[G])
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum(genstest[g, t] * GEN[g] for g,t in G)
    )
    
    # old Supply demand balances
    #@constraint(DCOPF, cBalance[i in N], 
     #   sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
      #      + sum(load for load in loads[loads.connnode .== i,:demand]) 
       # == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    #)

    # demand balance
    @constraint(DCOPF, cSupBalance[i in N],
        sum((GEN[g]) for g in gens[gens.connode .== i, :id]) 
        - sum((FLOW[l]) for l in lines[lines.fromnode .== i, :fromnode])
        == loads[loads.connnode .== i,:demand]
    )
            
    #demand balance reattempt? # need to sum over set G_i, sum over  set L_i
    #@constraint(DCOPF, cSBal,
     #       sum((I[g,N])*GEN[g]) for g in gens[])
      #      -sum((A[l,N])*FLOW[l] for l in lines[lines.id .== l])

    # Max generation constraint # AMMENDED 
    #@constraint(DCOPF, cMaxGen[g in G],
     #         gens[g,:pgmin]*U[G] <= GEN[g] <= gens[g,:pgmax]*U[G])
            
     # Max generation constraint # AMMENDED 
    @constraint(DCOPF, cMaxGen[g in G],
              GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch #AMMENDED
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode]] == 
            lines[l,:b] * sum((THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])) 
            
    )
    
    # line flow constraints AMMENDED
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            -lines[l,:capacity] <= FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    #Ammmended to show flows on each line 
    flows = DataFrame(
        flow = lines.b .* sum((angles[lines.fromnode] .- 
                        angles[lines.tonode]) * A[L,N] ))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
   
    # Return load payment
    loadpay = sum((dual.(cSupBalance)) * loads[loads.connnode .== i,:demand])
    
    #congestion cost
    congestc = -sum(FLOW[l] for l in load )

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
endd

In [41]:
datadir = joinpath("testcase") 
gens = CSV.read(joinpath(datadir,"Gen36.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran36_b_csv.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load36_csv.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

#lines2 = copy(lines)
#lines2.f = lines2.fromnode
#lines2.fromnode = lines.tonode
#lines2.tonode = lines.fromnode
#lines2 = lines2[:,names(lines)]
#append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.00266925,0.0323779,1,10000,1,30.8853
2,1,5,0.00287887,0.0286089,1,10000,2,34.9541
3,2,3,0.00131306,0.0190229,1,10000,3,52.5681
4,2,6,0.000803188,0.0149712,1,10000,4,66.795
5,3,10,0.000399063,0.00548875,1,10000,5,182.191
6,4,5,0.00114981,0.0183772,1,10000,6,54.4153
7,4,6,0.00209787,0.0318498,1,10000,7,31.3974
8,4,14,0.00220469,0.0290109,1,10000,8,34.4698
9,5,6,0.000706313,0.00961025,1,10000,9,104.056
